In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import llm_calls # all the interesting code is in here 
import parsing # all the interesting code is in here 
import openai
import os

In [3]:
# Add novels to this list
books = ['the_stranger.pdf', 'The-Bell-Jar-1645639705._vanilla.xml', 'franz-kafka_metamorphosis.epub']
parsed_books = [parsing.parser(book) for book in books] # parse book and split each book by chapters
coalesed_books = [parsing.coalesce(book) for book in parsed_books] # combine smaller chapters and information into one element

/Users/rohanpenmatcha/Library/Python/3.9/lib/python/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/Users/rohanpenmatcha/Library/Python/3.9/lib/python/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/rohanpenmatcha/Library/Python/3.9/lib/python/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you re

In [ ]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY") # If environmental variable isn't set up, you can replace the right hand side with the api key.
client = openai.OpenAI(api_key=OPENAI_API_KEY)
modelName = 'gpt-4o'

In [ ]:
chunk_level = []
outer = 1

for book in coalesed_books:
    temp = []
    for i, chapter in enumerate(book):
        temp.append(llm_calls.LiteraryAnalyst(chapter, modelName, client)) # Generate an Analysis on the chapter level for each book
        print(f"Finished iteration {i} \n")
    print(f"Finished Book {outer} \n")
    chunk_level.append(temp)
    outer += 1

novel_level = []
outer = 1
for novel in chunk_level:
    novel_level.append(llm_calls.SummarizeNovelLevelAnalysis(novel, modelName, client)) # Synthesize all the chunk-level analyses into one
    print(f"Finished Summarizing Book {outer} \n")
    outer += 1

comparison = llm_calls.CompareBooks(novel_level, modelName, client) # Comparative Analysis of the novel-level analysis (first time we see different texts )
print(f"Finished Comparing The Three Books \n")


# Generate the Body Paragraphs first -> Introduction (with a strong emphasis on thesis) -> Conclusion

body_paragraphs = llm_calls.GenerateBodyParagraphs(comparison, modelName, client) # Pass in Comparison
introduction = llm_calls.GenerateIntroduction(body_paragraphs, comparison, modelName, client) # Pass in Previous and Body Paragraph
conclusion = llm_calls.GenerateConclusion(introduction, body_paragraphs, comparison, modelName, client) # Pass in Previous and Introduction

result = '\n'.join([introduction, body_paragraphs, conclusion])
# result = re.sub(r'\(body paragraph(?: \d+(?:, \s?\d+)*)?\)', '', result)


with open("book_report_final_output.docx", "w") as file:
    parsing.save_to_doc(result, 'book_report_final_output.docx')